# 0.导入必要的库

In [10]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import re
import numpy as np
import pandas as pd

# 不再需要 import sys, import os (除非其他地方用到)
# 也不再需要任何 sys.path.append(...)

# 初始化GEE
ee.Initialize(project='geemap-441216') # 替换为您的项目ID

# Python会自动在当前文件夹下寻找模块，所以可以直接导入！
try:
    import z_flood_robust
    from z_flood_robust import zscore # 甚至可以这样导入
    zscore.calc_basemad # 尝试访问
    print("本地自定义模块 z_flood_robust 已成功导入！")
except ImportError as e:
    print(f"导入模块失败: {e}")
    print("请确保'z_flood_robust'文件夹与您的Jupyter笔记本在同一个目录下。")

本地自定义模块 z_flood_robust 已成功导入！


In [11]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle #用于在图表中添加形状(例如矩形)
import re #正则表达式模块
from z_flood_robust import calc_basemedian, calc_basemad, calc_median_anomaly, calc_robust_zscore, calc_basemean,calc_basestd,calc_zscore
from z_flood_robust import mapFloods,floodPalette

from ipywidgets import Label #用于创建交互式控件

In [12]:
# 定义全局状态变量
# 这些变量用于在不同分析步骤之间传递关键数据
roi_global = None
rz_image_global = None
sar_image_global = None

# 各个分析步骤的产出物
global_water_binary = None #二值化影像
buffered_edges_global = None 
filtered_bboxes_global = None
final_features_with_thresholds_list = []
final_flood_map_global = None


# 1.Initialize Map and UI

In [13]:
Map = geemap.Map(center=[39.4,-0.4],zoom=10,layout={'height':'600px'})


In [14]:
# step1:set parameter
target_date_picker = widgets.DatePicker(description='Flood Target_date:',value=datetime.date(2024,10,31))
baseline_start_picker = widgets.DatePicker(description='baseline_start:', value=datetime.date(2023, 11, 20))
baseline_end_picker = widgets.DatePicker(description='baseline_end:', value=datetime.date(2024, 9, 10))
# Button Control
load_s1_button = widgets.Button(description="1. 加载数据并计算RZ", button_style='primary', icon='cloud-download-alt')
# Textbox
point_click_label = widgets.Label("提示：请先在地图上使用左侧工具栏的矩形工具绘制一个研究区(ROI)。")

# step2:Flood analysis(all button banned originally)
otsu_button = widgets.Button(description="2.1 OTSU二值化", icon='adjust', disabled=True)
edge_buffer_button = widgets.Button(description="2.2 边缘缓冲区提取", icon='vector-square', disabled=True)
bbox_profile_button = widgets.Button(description="2.3 外包矩形筛选", icon='search', disabled=True)
rz_flood_button = widgets.Button(description="2.4 生成最终洪水图", button_style='warning', icon='tint', disabled=True)

# step3:Damage assessment
assess_pop_button = widgets.Button(description="可视化受灾人口", icon='users', disabled=True)
assess_crop_button = widgets.Button(description="可视化受灾农田", icon='leaf', disabled=True)
assess_built_button = widgets.Button(description="可视化受灾建成区", icon='building', disabled=True)

# management tools
clear_layers_button = widgets.Button(description="清除所有分析图层", button_style='danger', icon='trash')
output_console = widgets.Output(layout={'border': '1px solid black', 'height': '100px', 'overflow_y': 'scroll', 'width':'98%'})
# Output() create a container print result message

# 2.Core Function

In [15]:
# [helper function] remove Map
def clear_map_layers():
    while len(Map.layers) > 1:Map.remove_layer(Map.layers[-1])
    Map.remove_legend()
    
# [helper function] OTSU algorithm
def otsu(histogram):
  histogram = ee.Dictionary(histogram);
  counts = ee.Array(histogram.get('histogram'))
  means = ee.Array(histogram.get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  indices = ee.List.sequence(1,size)
  def function(i):
    aCounts = counts.slice(0,0,i); aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0,0,i); aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCount)
    bCount = total.subtract(aCount); bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(bCount.multiply(bMean.subtract(mean).pow(2)))
  bss = indices.map(function)
  return means.sort(bss).get([-1])

# [helper function] Rectangle Filter
def analyze_and_filter_bbox_by_outlier_percentage(feature, rz_image_for_analysis):
    try:
        OUTLIER_RZ_THRESHOLD = -2.0; MIN_OUTLIER_PERCENTAGE = 0.1
        error_margin = ee.ErrorMargin(1); bbox_geom = feature.geometry()
        center = bbox_geom.centroid(error_margin); coords = ee.List(bbox_geom.coordinates().get(0))
        p1, p2, p3 = ee.List(coords.get(0)), ee.List(coords.get(1)), ee.List(coords.get(2))
        p1_coords, p2_coords, p3_coords = np.array(p1.getInfo()), np.array(p2.getInfo()), np.array(p3.getInfo())
        center_coords = np.array(center.coordinates().getInfo())
        vector_12, vector_23 = p2_coords - p1_coords, p3_coords - p2_coords
        main_axis_vector = vector_12 if np.linalg.norm(vector_12) < np.linalg.norm(vector_23) else vector_23
        start_point = center_coords - main_axis_vector * 1.5; end_point = center_coords + main_axis_vector * 1.5
        long_line = ee.Geometry.LineString([start_point.tolist(), end_point.tolist()])
        profile_line = long_line.intersection(bbox_geom, error_margin)
        sampled_dict = rz_image_for_analysis.select('VV').reduceRegion(reducer=ee.Reducer.toList(), geometry=profile_line, scale=10, maxPixels=1024)
        rz_values = ee.List(sampled_dict.get('VV')).getInfo()
        if not rz_values or len(rz_values) < 3: return feature.set({'is_valid': 0, 'reason': 'NotEnoughPoints'})
        rz_array = np.array(rz_values); outlier_count = np.sum(rz_array < OUTLIER_RZ_THRESHOLD)
        outlier_percentage = outlier_count / len(rz_array)
        is_valid_python = outlier_percentage >= MIN_OUTLIER_PERCENTAGE
        return feature.set({'is_valid': 1 if is_valid_python else 0, 'outlier_percentage': outlier_percentage, 'profile_line': profile_line})
    except Exception as e: return feature.set({'is_valid': 0, 'reason': f'Error: {str(e)}'})
    
# [helper function] calculate threshold
def calculate_threshold_and_get_key_points(feature, rz_image_for_analysis):
    try:
        VALLEY_POINT_THRESHOLD = -1.0
        profile_line = ee.Geometry(feature.get('profile_line'))
        if profile_line is None: return feature.set({'local_threshold': -999})
        lon_lat_image = ee.Image.pixelLonLat()
        rz_dict = rz_image_for_analysis.select('VV').reduceRegion(reducer=ee.Reducer.toList(), geometry=profile_line, scale=10)
        lon_dict = lon_lat_image.select('longitude').reduceRegion(reducer=ee.Reducer.toList(), geometry=profile_line, scale=10)
        lat_dict = lon_lat_image.select('latitude').reduceRegion(reducer=ee.Reducer.toList(), geometry=profile_line, scale=10)
        rz_values, longitudes, latitudes = ee.List(rz_dict.get('VV')).getInfo(), ee.List(lon_dict.get('longitude')).getInfo(), ee.List(lat_dict.get('latitude')).getInfo()
        if not rz_values or len(rz_values) < 3: return feature.set({'local_threshold': -999})
        rz_array = np.array(rz_values); gradients = np.gradient(rz_array)
        valley_points = np.minimum(rz_array[:-1], rz_array[1:])
        qualified_indices = np.where(valley_points < VALLEY_POINT_THRESHOLD)[0] #extract index
        if len(qualified_indices) < 2: return feature.set({'local_threshold': -999})
        qualified_gradients = gradients[qualified_indices]
        if len(qualified_gradients) == 0: return feature.set({'local_threshold': -999})
        local_idx_max = np.argmax(qualified_gradients); local_idx_min = np.argmin(qualified_gradients)
        original_idx_max_grad = qualified_indices[local_idx_max]; original_idx_min_grad = qualified_indices[local_idx_min]
        # determine the key point 
        # determine how calculate threshold
        rz_at_neg_grad_point = valley_points[original_idx_min_grad]; rz_at_pos_grad_point = valley_points[original_idx_max_grad] #最正与最负梯度点
        rz_at_pos_gradfront_point = valley_points[original_idx_max_grad+1]
        local_threshold = (rz_at_pos_gradfront_point + rz_at_pos_grad_point) / 2.0
        
        neg_valley_idx = original_idx_min_grad if rz_array[original_idx_min_grad] < rz_array[original_idx_min_grad+1] else original_idx_min_grad + 1
        pos_valley_idx = original_idx_max_grad if rz_array[original_idx_max_grad] < rz_array[original_idx_max_grad+1] else original_idx_max_grad + 1
        neg_grad_point_geom = ee.Geometry.Point([longitudes[neg_valley_idx], latitudes[neg_valley_idx]])
        pos_grad_point_geom = ee.Geometry.Point([longitudes[pos_valley_idx], latitudes[pos_valley_idx]])
        return feature.set({'local_threshold': local_threshold, 'neg_grad_point': neg_grad_point_geom, 'pos_grad_point': pos_grad_point_geom})
    except Exception as e: return feature.set({'local_threshold': -999, 'error_msg': str(e)})


# 3.Button Callback Function

In [18]:
# [button load data] LOAD DATA 
# * args ->flexiable turple
# [最终融合版] 按钮 1 的回调函数 - 融合了您的原始逻辑和稳健性修复
def run_load_s1_step(*args):
    global roi_global, rz_image_global, sar_image_global

    if Map.user_roi is None:
        with output_console: clear_output(wait=True); print("错误：请先在地图上使用矩形工具绘制一个研究区(ROI)！")
        return
        
    roi_global = Map.user_roi
    targdate = target_date_picker.value.strftime('%Y-%m-%d')
    basestart = baseline_start_picker.value.strftime('%Y-%m-%d')
    baseend = baseline_end_picker.value.strftime('%Y-%m-%d')
    
    with output_console: clear_output(wait=True); print(f"研究区已确认。开始加载数据...\n目标日期: {targdate}, 基线: {basestart} to {baseend}")
    
    try:
        # --- 1. 定义一个足够宽的时间范围加载所有可能的数据 ---
        # 这样做效率更高，避免多次调用filterDate
        s1_collection_full = ee.ImageCollection("COPERNICUS/S1_GRD") \
            .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
            .filter(ee.Filter.equals("instrumentMode", "IW")) \
            .filter(ee.Filter.equals("orbitProperties_pass", "ASCENDING")) \
            .filterBounds(roi_global) \
            .filterDate(basestart, ee.Date(targdate).advance(15, 'day')) # 扩大窗口以备后用

        s1_size = s1_collection_full.size().getInfo()
        if s1_size == 0:
            with output_console: print("警告：在指定参数下未找到任何S1影像。"); return
        
        # --- 2. 尝试您的首选方法：在小窗口内合成 ---
        target_date_ee = ee.Date(targdate)
        date_window_days = 2 # 您可以调整这个窗口大小, 2代表目标日当天和后一天
        start_display_window = target_date_ee
        end_display_window = target_date_ee.advance(date_window_days, 'day')
        
        # 从完整集合中筛选出小窗口的数据
        s1_in_window = s1_collection_full.filterDate(start_display_window, end_display_window)
        
        # 尝试合成
        target_image_mosaic = s1_in_window.mosaic().clip(roi_global)
        
        # --- 3. 检查首选方法是否成功，如果不成功，启动备用方案 ---
        if target_image_mosaic.bandNames().size().getInfo() == 0:
            with output_console:
                print(f"警告: 在 {date_window_days} 天的窗口内未找到影像。正在启动备用方案...")
                print("正在寻找离目标日期最近的一张有效影像...")

            # [备用方案] 寻找最近的影像
            def add_time_diff(image):
                return image.set('time_diff', ee.Number(image.get('system:time_start')).subtract(target_date_ee.millis()).abs())
            s1_with_diff = s1_collection_full.filterDate(basestart, ee.Date(targdate).advance(15, 'day')).map(add_time_diff)
            sar_image_global = ee.Image(s1_with_diff.sort('time_diff').first()).clip(roi_global)
            actual_date_str = ee.Date(sar_image_global.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            info_message = f"已自动选择最近的S1影像 (日期: {actual_date_str})"
        else:
            # [首选方案成功] 直接使用合成的影像
            sar_image_global = target_image_mosaic
            info_message = f"已成功合成目标日期窗口内的S1影像"

        # --- 4. 计算 RZ-Score (使用相同的逻辑) ---
        s1_for_rz_calc = s1_collection_full.filterDate(basestart, ee.Date(targdate).advance(15, 'day'))
        rz_collection = calc_robust_zscore(s1_for_rz_calc, basestart, baseend, 'IW', "ASCENDING")
        
        # 同样，稳健地找到与最终选定的sar_image_global日期最匹配的RZ影像
        rz_image_global = rz_collection.filterDate(ee.Date(targdate),ee.Date(targdate).advance(2,'day'))\
                                         .mosaic().clip(roi_global)

        # --- 5. 可视化与反馈 ---
        Map.addLayer(sar_image_global.select('VV'), {'min': -25, 'max': 0}, '目标期 S1 VV')
        Map.addLayer(rz_image_global.select('VV'), {'min': -5, 'max': 5, 'palette': ['blue','white','red']}, '目标期 RZ-score')
        Map.centerObject(roi_global, 11)
        
        with output_console:
            print(f"成功加载 {s1_size} 张S1影像。")
            print(info_message) # 打印出是哪种方案成功了
            print("RZ-Score和目标日SAR影像已准备就绪。")
            print("下一步：请点击“2.1 OTSU二值化”。")

        # 激活下一步按钮
        otsu_button.disabled = False
        edge_buffer_button.disabled = True; bbox_profile_button.disabled = True; rz_flood_button.disabled = True
        assess_pop_button.disabled = True; assess_crop_button.disabled = True; assess_built_button.disabled = True

    except Exception as e:
        with output_console: print(f"数据加载失败: {e}")
        
        
# [button otsu]
def run_otsu_step(*args):
    global global_water_binary
    with output_console: clear_output(wait=True); print("--- 步骤 2.1: 执行OTSU二值化与形态学开运算 ---")
    try:
        globalThreshold = ee.Number(-13.5) # 使用经验证的阈值
        globalWater = sar_image_global.select('VV').lt(globalThreshold)
        kernel = ee.Kernel.circle(radius=2)
        opened_water = globalWater.focal_min(kernel=kernel, iterations=1).focal_max(kernel=kernel, iterations=1)
        global_water_binary = opened_water.selfMask()
        
        Map.addLayer(global_water_binary, {'palette': 'cyan'}, 'OTSU二值化结果')
        with output_console:
            print("OTSU二值化和开运算完成。")
            print("下一步：请点击“2.2 边缘缓冲区提取”。")
        edge_buffer_button.disabled = False
    except Exception as e:
        with output_console: print(f"OTSU步骤失败: {e}")
        
# [button buffer extract]
def run_edge_buffer_step(*args):
    global buffered_edges_global
    with output_console: clear_output(wait=True); print("--- 步骤 2.2: 执行边缘检测与缓冲区生成 ---")
    try:
        canny = ee.Algorithms.CannyEdgeDetector(image=global_water_binary, threshold=1, sigma=1)
        connected = canny.updateMask(canny).lt(0.05).connectedPixelCount(100,True)
        edges = connected.gte(50)
        buffered_edges_global = edges.fastDistanceTransform().lt(15)
        
        Map.addLayer(buffered_edges_global.selfMask(), {'palette': 'yellow', 'opacity': 0.7}, '边缘缓冲区')
        with output_console:
            print("边缘检测和缓冲区提取完成。")
            print("下一步：请点击“2.3 外包矩形筛选”。")
        bbox_profile_button.disabled = False
    except Exception as e:
        with output_console: print(f"边缘缓冲区步骤失败: {e}")
    
# 【button filter+display bounding rec]
def run_bbox_profile_step(*args):
    global filtered_bboxes_global
    with output_console:clear_output(wait = True);print("--step 2.3:generate+filter bounding rec---")
    try:
        bufferVectors = buffered_edges_global.selfMask().reduceToVectors(
            geometry = roi_global,
            scale = 10,
            eightConnected = True,
            maxPixels = 1e10
        )
        # ---修复：在map函数中，返回ee.Feature 而不是 ee.Geometry---
        def get_bounding_box_as_feature(feature):
            bbox_geometry = feature.geometry().bounds()
            return ee.Feature(bbox_geometry)
        
        unfiltered_bboxes = bufferVectors.map(get_bounding_box_as_feature) 
        # Client-Side loop filter
        unfiltered_list = unfiltered_bboxes.toList(unfiltered_bboxes.size())
        filtered_list = []
        for i in range(unfiltered_list.size().getInfo()):
            feature = ee.Feature(unfiltered_list.get(i))
            processed_feature = analyze_and_filter_bbox_by_outlier_percentage(feature,rz_image_global)
            if processed_feature.getInfo()['properties']['is_valid'] == 1:
                filtered_list.append(processed_feature)
        filtered_bboxes_global = ee.FeatureCollection(filtered_list)
        
        # paint rec
        outline_unfiltered = ee.Image().byte().paint(unfiltered_bboxes,0,1)
        outline_filtered = ee.Image().byte().paint(filtered_bboxes_global,0,3)
        
        # visualize
        Map.addLayer(outline_filtered,{'palette':'lime'},'筛选后的有效矩形')
        with output_console:
            print(f"共生成 {unfiltered_bboxes.size().getInfo()} 个外包矩形，筛选后剩余 {filtered_bboxes_global.size().getInfo()} 个。")
            print("下一步：请点击“2.4 生成最终洪水图”。")
        rz_flood_button.disabled = False
    except Exception as e:
        with output_console: print(f"外包矩形筛选步骤失败: {e}")
        
        
# [button threshold rz-flood]  
def run_rz_flood_step(*args):
    global final_flood_map_global, final_features_with_thresholds_list
    with output_console: clear_output(wait=True); print("--- Step2.4:Calculate local threshold and generate flood  ---")
    try:
        filtered_list = filtered_bboxes_global.toList(filtered_bboxes_global.size())
        final_features_with_thresholds_list = []
        for i in range(filtered_list.size().getInfo()):
            feature = ee.Feature(filtered_list.get(i))
            feature_with_results = calculate_threshold_and_get_key_points(feature, rz_image_global)
            if feature_with_results.getInfo()['properties']['local_threshold'] != -999:
                final_features_with_thresholds_list.append(feature_with_results)
        if not final_features_with_thresholds_list:
            with output_console: print("警告：未能成功计算任何局部阈值。流程终止。"); return
        final_results_fc = ee.FeatureCollection(final_features_with_thresholds_list)
        local_thresholds_py_list = final_results_fc.aggregate_array('local_threshold').getInfo()
        global_average_threshold = np.mean(local_thresholds_py_list)
        def apply_local_threshold(feature):
            local_thresh = ee.Number(feature.get('local_threshold'))
            return rz_image_global.lt(local_thresh).And(ee.Image.constant(1).clip(feature.geometry())).selfMask()
        all_local_floods = ee.ImageCollection(final_results_fc.map(apply_local_threshold)).mosaic()
        potential_global_flood = rz_image_global.lt(global_average_threshold)
        all_local_floods_unmasked = all_local_floods.unmask(0)
        intra_region_mask = all_local_floods.mask().Not().Not()
        extra_regional_flood_only = potential_global_flood.where(intra_region_mask, 0)
        final_flood_map_global = extra_regional_flood_only.add(all_local_floods_unmasked)
        Map.addLayer(final_flood_map_global.selfMask(), {'palette': 'blue'}, '最终洪水范围')
        with output_console:
            print(f"成功为 {len(final_features_with_thresholds_list)} 个矩形计算了局部阈值。")
            print(f"全局平均阈值为: {global_average_threshold:.2f}")
            print("最终洪水图生成完毕！")
            print("下一步：可以执行“第三步：洪灾损失评估”。")
        assess_pop_button.disabled = False; assess_crop_button.disabled = False; assess_built_button.disabled = False
    except Exception as e:
        with output_console: print(f"生成最终洪水图步骤失败: {e}")
        
# [button damage assess]
# *args -> impact_type(pop,farmland,constructiom)
def assess_impact_func(impact_type):
    if final_flood_map_global is None:
        with output_console: clear_output(wait=True); print("Warning:Generate flood first(step2.4)。"); return
    with output_console: clear_output(wait=True); print(f"--- 步骤 3: 开始评估 {impact_type} 受灾情况 ---")  
    try:
        flood_mask = final_flood_map_global.eq(1)
        scale = 100
        if impact_type == "pop":
            dataset = ee.ImageCollection('JRC/GHSL/P2023A/GHS_POP').mosaic().clip(roi_global)
            stats_reducer = ee.Reducer.sum()
            vis_params = {'min': 0, 'max': 500, 'palette': ['lightyellow', 'orange', 'red']}
            unit = "人"
            factor = 1
        elif impact_type == "farmland":
            dataset = ee.ImageCollection("ESA/WorldCover/v200").first().select('Map').eq(40).multiply(ee.Image.pixelArea())
            stats_reducer = ee.Reducer.sum()
            vis_params = {'palette': ['#006400']}
            unit = "平方公里"
            factor = 1e6
            scale = 10
        elif impact_type == "construction":
            dataset = ee.ImageCollection("ESA/WorldCover/v200").first().select('Map').eq(50).multiply(ee.Image.pixelArea())
            stats_reducer = ee.Reducer.sum()
            vis_params = {'palette': ['#C0C0C0']}
            unit = "平方公里"
            factor = 1e6
            scale = 10
        affected_image = dataset.updateMask(flood_mask)
        Map.addLayer(affected_image, vis_params, f'受灾{impact_type}区域')
        stats = affected_image.reduceRegion(reducer=stats_reducer, geometry=roi_global, scale=scale, maxPixels=1e10)
        band_name = dataset.bandNames().get(0).getInfo()
        affected_value = stats.get(band_name).getInfo()
        with output_console:
            print(f"{impact_type} 影响评估完成。")
            print(f"估计受灾{impact_type}: {affected_value / factor:,.2f} {unit}")
    except Exception as e:
        with output_console: print(f"评估 {impact_type} 影响时出错: {e}")
    
#[button clear all layer]
def on_clear_layers_button_clicked(*args):
    with output_console: clear_output(wait=True); print("正在清除所有分析图层...")
    clear_map_layers()
    otsu_button.disabled = True; edge_buffer_button.disabled = True; bbox_profile_button.disabled = True; rz_flood_button.disabled = True
    assess_pop_button.disabled = True; assess_crop_button.disabled = True; assess_built_button.disabled = True
    with output_console: print("Analysis layers cleared,process reset。")

# 4.Event Binding

In [19]:
load_s1_button.on_click(run_load_s1_step)
otsu_button.on_click(run_otsu_step)
edge_buffer_button.on_click(run_edge_buffer_step)
bbox_profile_button.on_click(run_bbox_profile_step)
rz_flood_button.on_click(run_rz_flood_step)
assess_pop_button.on_click(lambda b: assess_impact_func("pop"))
assess_crop_button.on_click(lambda b: assess_impact_func("farmland"))
assess_built_button.on_click(lambda b: assess_impact_func("construction"))
clear_layers_button.on_click(on_clear_layers_button_clicked)

# 5.Construct and display UI layout

In [20]:
step1_box = widgets.VBox([target_date_picker,baseline_start_picker,baseline_end_picker,load_s1_button,point_click_label])

step2_box = widgets.VBox([otsu_button,edge_buffer_button,bbox_profile_button,rz_flood_button])

step3_box = widgets.VBox([widgets.Label("Assess various flood damagement"),widgets.HBox([assess_pop_button,assess_crop_button,assess_built_button])])

#Accordion:a vertically stacked menu only allows one container unfold
accordion = widgets.Accordion(children=[step1_box,step2_box,step3_box])
accordion.set_title(0, '第一步：参数设置与数据加载'); accordion.set_title(1, '第二步：洪水范围提取'); accordion.set_title(2, '第三步：洪灾损失评估')
ui_layout = widgets.VBox([accordion, widgets.HTML("<h3>管理工具:</h3>"), clear_layers_button, widgets.HTML("<h3>日志输出:</h3>"), output_console])

# ultimate display
with output_console:
    print("欢迎使用交互式洪水分析系统！")
    print("1. 在地图上使用左侧工具栏的矩形工具绘制一个您感兴趣的研究区(ROI)。")
    print("2. 设置日期，然后点击“加载数据并计算RZ”按钮开始分析。")
display(ui_layout, Map)

Map(center=[39.4, -0.4], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', …